# Day Trading Strategy Analysis

This notebook provides interactive analysis of day trading strategies using technical indicators like Moving Averages, Bollinger Bands, RSI, and MACD.

## Setup and Data Loading

In [ ]:
import os
import sys
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import seaborn as sns

# Add the project root to the Python path
sys.path.append(os.path.abspath('..'))

# Import project modules
from data.data_loader import download_data, load_data, process_data_for_day_trading
from indicators.technical_indicators import sma, ema, bollinger_bands, rsi, macd
from strategy.day_trading_strategy import DayTradingStrategy
from backtest.backtester import DayTradingBacktester
from visualization.chart_visualizer import plot_price_with_indicators, plot_trades, plot_equity_curve, plot_performance_summary
from utils.analysis import analyze_trades, compare_timeframes
from utils.optimizer import StrategyOptimizer, optimize_strategy

# Set up plotting style
plt.style.use('seaborn-darkgrid')
plt.rcParams['figure.figsize'] = (15, 8)
sns.set_context("notebook")

# Display all columns
pd.set_option('display.max_columns', None)

## Load Configuration

In [ ]:
# Load the configuration file
def load_config(config_path='../config/config.json'):
    with open(config_path, 'r') as f:
        config = json.load(f)
    return config

config = load_config()
config

## Download or Load Market Data

In [ ]:
# Set parameters for data download
symbol = "AAPL"  # Change this to the symbol you want to analyze
start_date = "2023-01-01"
end_date = "2023-12-31"
timeframe = "15m"  # Options: '1m', '5m', '15m', '30m', '1h', '1d'

# Download or load data
data_dir = config['data']['data_dir']
cache_filename = f"{symbol}_{timeframe}_{start_date.replace('-', '')}_{end_date.replace('-', '')}.csv"
cache_path = os.path.join(data_dir, cache_filename)

if os.path.exists(cache_path) and config['data']['use_cached_data']:
    print(f"Loading cached data from {cache_path}")
    df = load_data(cache_path)
else:
    print(f"Downloading data for {symbol} from {start_date} to {end_date}")
    df = download_data(symbol, start_date, end_date, timeframe, True, data_dir)

# Process data for day trading if trading hours are specified
if 'trading_hours' in config['backtest']:
    start_hour = config['backtest']['trading_hours']['start']
    end_hour = config['backtest']['trading_hours']['end']
    df = process_data_for_day_trading(df, (start_hour, end_hour))

# Display basic information about the data
print(f"Data shape: {df.shape}")
print(f"Date range: {df.index[0]} to {df.index[-1]}")
df.head()

## Calculate Technical Indicators

In [ ]:
# Create a copy of the data
analysis_df = df.copy()

# Get indicator parameters from config
indicator_params = config['strategy']['indicators']

# Calculate Moving Averages
analysis_df['sma_fast'] = sma(analysis_df['close'], indicator_params['sma_fast_period'])
analysis_df['sma_slow'] = sma(analysis_df['close'], indicator_params['sma_slow_period'])
analysis_df['ema_fast'] = ema(analysis_df['close'], indicator_params['sma_fast_period'])
analysis_df['ema_slow'] = ema(analysis_df['close'], indicator_params['sma_slow_period'])

# Calculate Bollinger Bands
upper_band, middle_band, lower_band = bollinger_bands(
    analysis_df['close'], 
    indicator_params['bb_period'],
    indicator_params['bb_std']
)
analysis_df['bb_upper'] = upper_band
analysis_df['bb_middle'] = middle_band
analysis_df['bb_lower'] = lower_band

# Calculate RSI
analysis_df['rsi'] = rsi(analysis_df['close'], indicator_params['rsi_period'])

# Calculate MACD
macd_line, signal_line, histogram = macd(
    analysis_df['close'],
    indicator_params['macd_fast_period'],
    indicator_params['macd_slow_period'],
    indicator_params['macd_signal_period']
)
analysis_df['macd_line'] = macd_line
analysis_df['macd_signal'] = signal_line
analysis_df['macd_histogram'] = histogram

# Remove NaN values
analysis_df = analysis_df.dropna()

# Display the first few rows
analysis_df.head()

## Visualize Price and Indicators

In [ ]:
# Plot price with indicators
plot_price_with_indicators(analysis_df, title=f"{symbol} - Price Chart with Indicators")

## Run Backtest

In [ ]:
# Create strategy with parameters from config
strategy_config = {
    **config['strategy']['indicators'],
    **config['strategy']['risk_management']
}
strategy = DayTradingStrategy(strategy_config)

# Create backtester
initial_capital = config['backtest']['initial_capital']
backtester = DayTradingBacktester(strategy, initial_capital)

# Run backtest
results_df = backtester.run(analysis_df, timeframe)

# Get performance summary
performance = backtester.get_performance_summary()

# Display performance metrics
performance_df = pd.DataFrame({
    'Metric': [
        'Total Trades',
        'Winning Trades',
        'Losing Trades',
        'Win Rate (%)',
        'Average Win (%)',
        'Average Loss (%)',
        'Profit Factor',
        'Total Return (%)',
        'Max Drawdown (%)'
    ],
    'Value': [
        performance['total_trades'],
        performance['winning_trades'],
        performance['losing_trades'],
        performance['win_rate'] * 100,
        performance['avg_win'],
        performance['avg_loss'],
        performance['profit_factor'],
        performance['total_return'],
        performance['max_drawdown']
    ]
})

performance_df

## Visualize Trades and Equity Curve

In [ ]:
# Plot trades
plot_trades(results_df, title=f"{symbol} - {timeframe} Trades")

# Plot equity curve
plot_equity_curve(results_df, title=f"{symbol} - {timeframe} Equity Curve")

# Plot performance summary
plot_performance_summary(performance)

## Analyze Trades

In [ ]:
# Extract completed trades
trades_df = results_df[results_df['pnl'] != 0].copy()

# Display basic trade information
print(f"Total trades: {len(trades_df)}")
print(f"Winning trades: {len(trades_df[trades_df['pnl'] > 0])}")
print(f"Losing trades: {len(trades_df[trades_df['pnl'] < 0])}")

# Display trade statistics
trades_df[['signal', 'signal_reason', 'entry_price', 'exit_price', 'pnl']].head(10)

## Optimize Strategy Parameters

In [ ]:
# Define parameter grid for optimization
param_grid = {
    'rsi_period': [7, 14, 21],
    'rsi_oversold': [20, 30, 40],
    'rsi_overbought': [60, 70, 80],
    'sma_fast_period': [10, 20, 50],
    'sma_slow_period': [50, 100, 200],
    'bb_period': [10, 20, 30],
    'bb_std': [1.5, 2.0, 2.5],
    'stop_loss_pct': [0.5, 1.0, 2.0],
    'take_profit_pct': [1.0, 2.0, 3.0]
}

# Note: To actually run the optimization, uncomment and run this code
# It can take a long time depending on the number of parameter combinations

'''
# Define scoring function (higher is better)
def custom_score(performance):
    # Return based scoring with drawdown penalty
    return performance.get('total_return', 0) - 0.5 * abs(performance.get('max_drawdown', 0))

# Run optimization
best_params, best_score, optimizer = optimize_strategy(
    analysis_df, 
    param_grid, 
    timeframe=timeframe, 
    initial_capital=initial_capital,
    scoring=custom_score,
    max_combinations=50  # Limit number of combinations to test
)

print(f"Best score: {best_score}")
print(f"Best parameters: {best_params}")

# Get top strategies
top_strategies = optimizer.get_top_n_strategies(5)
for i, strategy in enumerate(top_strategies):
    print(f"\nRank {i+1}:")
    print(f"Score: {strategy['score']:.2f}")
    print(f"Parameters: {strategy['params']}")
    print(f"Performance: Win Rate: {strategy['performance']['win_rate']*100:.2f}%, Total Return: {strategy['performance']['total_return']:.2f}%")
'''

## Compare Different Timeframes

In [ ]:
# Note: This code shows how to compare different timeframes
# Uncomment and run if you have data for different timeframes

'''
# List of timeframes to compare
timeframes = ['15m', '1h']
comparison_results = {}

for tf in timeframes:
    # Load data for this timeframe
    cache_filename = f"{symbol}_{tf}_{start_date.replace('-', '')}_{end_date.replace('-', '')}.csv"
    cache_path = os.path.join(data_dir, cache_filename)
    
    if os.path.exists(cache_path):
        print(f"Loading cached data for {tf} timeframe")
        tf_df = load_data(cache_path)
    else:
        print(f"Downloading data for {tf} timeframe")
        tf_df = download_data(symbol, start_date, end_date, tf, True, data_dir)
    
    # Process data for day trading
    if 'trading_hours' in config['backtest']:
        start_hour = config['backtest']['trading_hours']['start']
        end_hour = config['backtest']['trading_hours']['end']
        tf_df = process_data_for_day_trading(tf_df, (start_hour, end_hour))
    
    # Run backtest
    backtester = DayTradingBacktester(strategy, initial_capital)
    results_df = backtester.run(tf_df, tf)
    performance = backtester.get_performance_summary()
    
    # Store results
    comparison_results[tf] = performance

# Compare timeframes
comparison_df = compare_timeframes({symbol: comparison_results})
comparison_df
'''

## Conclusion

This notebook has demonstrated how to:
1. Load and prepare market data
2. Calculate and visualize technical indicators
3. Backtest a day trading strategy
4. Analyze trading performance
5. Optimize strategy parameters
6. Compare different timeframes

For production use, you may want to:
1. Use a larger dataset for more reliable results
2. Implement walk-forward testing to validate strategy robustness
3. Add additional risk management rules
4. Consider transaction costs and slippage
5. Implement portfolio-level backtesting with multiple symbols